In [ ]:
import os
import gc
import joblib
import numpy as np
import polars as pl
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import pad_sequences, to_categorical

# --- Your existing function imports ---
from src.nn_blocks import (
    unet_se_cnn,
    features_processing, 
    GatedMixupGenerator, 
    tof_block, 
    match_time_steps, 
    time_sum, 
    squeeze_last_axis,
    expand_last_axis
)

from src.functions import (
    train_model, 
    create_sequence_dataset,
    perform_padding,
    generate_gate_targets
)
from src.constants import DATA_PATH

# =====================================================================================
# MASTER CONTROL FLAG
# =====================================================================================
TRAIN = True 
TRAIN = False

# =====================================================================================
# CONFIGURATION
# =====================================================================================
PARQUET_FILE = 'output/final_processed_train_data.parquet'
PRETRAINED_DIR = Path("output/artifacts")
PRETRAINED_DIR.mkdir(parents=True, exist_ok=True) # Ensure directory exists

LR_INIT = 5e-4
WD = 3e-3
NUM_CLASSES = 18
BATCH_SIZE = 64
N_SPLITS = 4 
MAX_PAD_LEN = 128

# =====================================================================================
# MODEL DEFINITION (Your existing function)
# =====================================================================================
def create_model(dataset, imu_dim, wd=1e-4):
    sample_batch = next(iter(dataset))
    input_shape = sample_batch[0].shape[1:]
    inp = tf.keras.layers.Input(shape=input_shape)
    imu = tf.keras.layers.Lambda(lambda t: t[:, :, :imu_dim])(inp)
    tof = tf.keras.layers.Lambda(lambda t: t[:, :, imu_dim:])(inp)

    x1 = unet_se_cnn(imu, 3, base_filters=64, kernel_size=3)
    x2 = tof_block(tof, wd)

    x = features_processing(x1, x2)
    x = tf.keras.layers.Dropout(0.3)(x) 
    main_out = tf.keras.layers.Dense(18, activation="softmax", name="main_output")(x)
    gate_out = tf.keras.layers.Dense(1, activation="sigmoid", name="tof_gate")(x) # Renamed layer
    
    return tf.keras.models.Model(inputs=inp, outputs={"main_output": main_out, "tof_gate": gate_out})

# =====================================================================================
# TRAINING LOGIC
# =====================================================================================
if TRAIN:
    schema_df = pl.read_parquet(PARQUET_FILE, n_rows=0)
    all_columns = schema_df.columns
    meta_cols = {'gesture', 'gesture_int', 'sequence_type', 'behavior', 'orientation',
                    'row_id', 'subject', 'phase', 'sequence_id', 'sequence_counter'}
    feature_cols = [c for c in all_columns if c not in meta_cols]
    imu_cols  = [c for c in feature_cols if not (c.startswith('thm_') or c.startswith('tof_'))]
    tof_cols  = [c for c in feature_cols if c.startswith('thm_') or c.startswith('tof_')]

    print("Scanning Parquet file for sequence IDs...")
    all_sequence_ids = (
        pl.scan_parquet(PARQUET_FILE)
        .select('sequence_id')
        .unique()
        .collect()
        .to_numpy()
        .ravel()
    )
    print(f"Found {len(all_sequence_ids)} unique sequences.")

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
    fold_accuracies = []
    all_preds = []
    all_labels = []
    imu_dim = len(imu_cols)

    for fold_idx, (train_indices, val_indices) in enumerate(kf.split(all_sequence_ids)):
        print(f"\n=== Fold {fold_idx + 1}/{N_SPLITS} ===")
        train_ids = all_sequence_ids[train_indices]
        val_ids = all_sequence_ids[val_indices]

        print(f"Loading data for fold {fold_idx + 1}...")
        train_df = pl.read_parquet(PARQUET_FILE).filter(pl.col('sequence_id').is_in(train_ids))
        val_df = pl.read_parquet(PARQUET_FILE).filter(pl.col('sequence_id').is_in(val_ids))
        print("Fold data loaded.")

        train_gate_df = generate_gate_targets(train_df, tof_cols)
        val_gate_df = generate_gate_targets(val_df, tof_cols)

        le = LabelEncoder()
        le.fit(train_df['gesture'])
        train_df = train_df.with_columns(pl.Series("gesture_int", le.transform(train_df['gesture'])))
        val_df = val_df.with_columns(pl.Series("gesture_int", le.transform(val_df['gesture'])))

        # --- StandardScaler Logic ---
        scaler = StandardScaler()
        # Fit on training data and transform both
        train_features_scaled = scaler.fit_transform(train_df[imu_cols + tof_cols])
        val_features_scaled = scaler.transform(val_df[imu_cols + tof_cols])
        # Create Polars DataFrames from the scaled numpy arrays
        X_train_scaled_features = pl.DataFrame(train_features_scaled, schema=imu_cols + tof_cols)
        X_val_scaled_features = pl.DataFrame(val_features_scaled, schema=imu_cols + tof_cols)

        meta_cols_to_keep = ['sequence_id', 'gesture_int']
        train_df_final = train_df.select(meta_cols_to_keep).with_columns(X_train_scaled_features)
        val_df_final = val_df.select(meta_cols_to_keep).with_columns(X_val_scaled_features)

        del train_df, val_df, X_train_scaled_features, X_val_scaled_features
        gc.collect()

        X_train, y_train, train_gate_target = create_sequence_dataset(train_df_final, imu_cols + tof_cols, train_gate_df)
        X_val, y_val, val_gate_target = create_sequence_dataset(val_df_final, imu_cols + tof_cols, val_gate_df)

        del train_df_final, val_df_final
        gc.collect()

        X_train_padded = perform_padding(X_train, MAX_PAD_LEN)
        X_val_padded = perform_padding(X_val, MAX_PAD_LEN)
        
        y_train_cat = to_categorical(y_train, num_classes=NUM_CLASSES)
        y_val_cat = to_categorical(y_val, num_classes=NUM_CLASSES)

        train_dataset = GatedMixupGenerator(
            X=X_train_padded, y=y_train_cat, gate_targets=train_gate_target,
            batch_size=BATCH_SIZE, imu_dim=imu_dim, alpha=0.2, masking_prob=0.25
        )
        val_dataset = tf.data.Dataset.from_tensor_slices((
            X_val_padded, {'main_output': y_val_cat, 'tof_gate': val_gate_target[:, np.newaxis]}
        )).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

        del X_val, y_val, X_train, y_train, X_train_padded, X_val_padded
        gc.collect()
        
        model = create_model(train_dataset, len(imu_cols))
        train_model(model, train_dataset, val_dataset, 150, LR_INIT, WD)

        # --- SAVE ARTIFACTS ---
        print(f"--- Saving artifacts for Fold {fold_idx + 1} ---")
        model.save(PRETRAINED_DIR / f"gesture_model_fold_{fold_idx}.h5")
        
        # Save scaler and other metadata only from the first fold
        if fold_idx == 0:
            joblib.dump(scaler, PRETRAINED_DIR / "scaler.pkl")
            np.save(PRETRAINED_DIR / "feature_cols.npy", np.array(imu_cols + tof_cols))
            np.save(PRETRAINED_DIR / "sequence_maxlen.npy", MAX_PAD_LEN)
            np.save(PRETRAINED_DIR / "gesture_classes.npy", le.classes_)
            print("Scaler, feature_cols, maxlen, and classes saved.")

        # --- EVALUATION ---
        val_preds = model.predict(val_dataset)
        main_output_preds = val_preds['main_output']
        y_pred_fold = np.argmax(main_output_preds, axis=1)
        y_true_fold = np.argmax(y_val_cat, axis=1)
        fold_acc = accuracy_score(y_true_fold, y_pred_fold)
        fold_accuracies.append(fold_acc)
        print(f"Fold {fold_idx + 1} Accuracy: {fold_acc:.4f}")
        all_preds.append(y_pred_fold)
        all_labels.append(y_true_fold)

        del train_dataset, model, val_dataset
        gc.collect()

    # --- FINAL OOF REPORT ---
    print("\n=== Cross-validation Summary ===")
    print(f"Per-fold Accuracies: {fold_accuracies}")
    print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")
    y_all_pred = np.concatenate(all_preds)
    y_all_true = np.concatenate(all_labels)
    print("\n=== Overall Classification Report ===")
    print(classification_report(y_all_true, y_all_pred, target_names=le.classes_, digits=4))

# =====================================================================================
# INFERENCE LOGIC
# =====================================================================================
else:
    import pandas as pd 
    from src.metric import CompetitionMetric 
    from tensorflow import argmax, minimum, shape

    def crop_or_pad(inputs):
        x, skip = inputs
        x_len = shape(x)[1]
        skip_len = shape(skip)[1]
        min_len = minimum(x_len, skip_len)
        return x[:, :min_len, :], skip[:, :min_len, :]
    
    # --- 1. Load All Inference Artifacts ---
    print("▶ INFERENCE MODE – loading artefacts from", PRETRAINED_DIR)
    final_feature_cols = np.load(PRETRAINED_DIR / "feature_cols.npy", allow_pickle=True).tolist()
    pad_len = int(np.load(PRETRAINED_DIR / "sequence_maxlen.npy"))
    scaler = joblib.load(PRETRAINED_DIR / "scaler.pkl")
    gesture_classes = np.load(PRETRAINED_DIR / "gesture_classes.npy", allow_pickle=True)

    models = []
    print(f"  Loading {N_SPLITS} models for ensemble inference...")
    for fold in range(N_SPLITS):
        model_path = PRETRAINED_DIR / f"gesture_model_fold_{fold}.h5"
        # Note: You only need to pass custom LAYERS/FUNCTIONS here, not the whole model definition
        model = load_model(model_path, compile=False, custom_objects={
            'unet_se_cnn': unet_se_cnn,
            'tof_block': tof_block,
            'features_processing': features_processing,
            'match_time_steps': match_time_steps,
            'crop_or_pad': crop_or_pad,
            'squeeze_last_axis': squeeze_last_axis,
            'expand_last_axis': expand_last_axis,
            'time_sum': time_sum
        })
        models.append(model)
    print("  Models, scaler, and metadata loaded – ready for evaluation.")

    # --- 2. Define TTA Parameters and Predict Function ---
    TTA_STEPS = 10
    TTA_NOISE_STDDEV = 0.01

    from src.tof_feats import remove_gravity_from_acc, calculate_angular_velocity_from_quat, calculate_angular_distance

    def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
        df_seq = sequence.to_pandas()
        
        # --- Feature Engineering (must match training) ---
        linear_accel = remove_gravity_from_acc(df_seq, df_seq)
        df_seq['linear_acc_x'], df_seq['linear_acc_y'], df_seq['linear_acc_z'] = linear_accel[:, 0], linear_accel[:, 1], linear_accel[:, 2]
        df_seq['linear_acc_mag'] = np.sqrt(df_seq['linear_acc_x']**2 + df_seq['linear_acc_y']**2 + df_seq['linear_acc_z']**2)
        df_seq['linear_acc_mag_jerk'] = df_seq['linear_acc_mag'].diff().fillna(0)
        angular_vel = calculate_angular_velocity_from_quat(df_seq)
        df_seq['angular_vel_x'], df_seq['angular_vel_y'], df_seq['angular_vel_z'] = angular_vel[:, 0], angular_vel[:, 1], angular_vel[:, 2]
        df_seq['angular_distance'] = calculate_angular_distance(df_seq)
        for i in range(1, 6):
            pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]
            tof_data = df_seq[pixel_cols].replace(-1, np.nan)
            df_seq[f'tof_{i}_mean'] = tof_data.mean(axis=1)
            df_seq[f'tof_{i}_std'] = tof_data.std(axis=1)
            df_seq[f'tof_{i}_min'] = tof_data.min(axis=1)
            df_seq[f'tof_{i}_max'] = tof_data.max(axis=1)
            
        mat_unscaled = df_seq[final_feature_cols].ffill().bfill().fillna(0).values.astype('float32')
        mat_scaled = scaler.transform(mat_unscaled)
        pad_input = pad_sequences([mat_scaled], maxlen=pad_len, padding='post', truncating='post', dtype='float32')

        # --- TTA Loop ---
        all_tta_predictions = []
        for i in range(TTA_STEPS):
            noisy_input = pad_input
            if i > 0: # First pass is always on clean data
                noise = tf.random.normal(shape=tf.shape(pad_input), mean=0.0, stddev=TTA_NOISE_STDDEV)
                noisy_input = pad_input + noise

            # Ensemble predictions from all fold models
            all_fold_predictions = []
            for model in models:
                main_preds, _ = model.predict(noisy_input, verbose=0)
                all_fold_predictions.append(main_preds)
            
            avg_fold_prediction = np.mean(all_fold_predictions, axis=0)
            all_tta_predictions.append(avg_fold_prediction)

        # --- Final Averaging and Prediction ---
        final_avg_prediction = np.mean(all_tta_predictions, axis=0)
        idx = int(final_avg_prediction.argmax())
        
        return str(gesture_classes[idx])

    # --- 3. Run Kaggle Evaluation Server ---
    import kaggle_evaluation.cmi_inference_server
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        # For local testing, you need to provide the paths to the test data
        print("Running local gateway for testing...")
        inference_server.run_local_gateway(
            data_paths=(
                'input/cmi-detect-behavior-with-sensor-data/test.csv',
                'input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
            )
        )

2025-08-10 00:20:40.601671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754781640.621021    6477 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754781640.626572    6477 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754781640.642059    6477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754781640.642085    6477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754781640.642087    6477 computation_placer.cc:177] computation placer alr

▶ INFERENCE MODE – loading artefacts from output/artifacts
  Loading 4 models for ensemble inference...


I0000 00:00:1754781643.947011    6477 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4714 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1


  Models, scaler, and metadata loaded – ready for evaluation.
Running local gateway for testing...


/home/uallende/projects/kaggle/CMI/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
I0000 00:00:1754781650.838803    6586 cuda_dnn.cc:529] Loaded cuDNN version 90300


GatewayRuntimeError: (<GatewayRuntimeErrorType.SERVER_RAISED_EXCEPTION: 3>, "the resolved dtypes are not compatible with add.reduce. Resolved (dtype('<U11'), dtype('<U11'), dtype('<U22'))")